In [ ]:
# from gensim import corpora, models, similarities
from collections import Counter
import os
from nltk.corpus import stopwords

# read all documents in one variant
path = 'f:\\nlp\\en\\COCA\\COCA_sample_lineartext\\'
documents = []
for filename in os.listdir(path+'result\\'):
    with open(path+'result\\'+filename) as f:
        documents += f.read().split('\n')

# remove common words and tokenize
stopwords = set(stopwords.raw('english').split()) ## 获取英文停用词表
texts = [[word for word in document.split() if word not in stopwords]
         for document in documents]

# remove words that appear only once
all_tokens = [word for text in texts for word in text]
freq = Counter(all_tokens)
tokens_once = set(word for word in fd if fd[word] == 1)
texts = [[word for word in text if word not in tokens_once]
         for text in texts]

# get th dictionary
allTokens = set(word for word in fd if fd[word] > 1)

In [20]:
def DictInit(keys,value):
    Dict = {}
    for k in keys:
        Dict[k] = value
    return Dict

In [22]:
import math

# conditional freq distribution and MI (using complete sentence)
CFD_sent = DictInit(allTokens,Counter())
MI_sent = DictInit(allTokens,{})

for sent in texts:
    for (index,token) in enumerate(sent):
        CFD_sent[token].update(sent)

wordsCount = sum([len(text) for text in texts])

for t1 in CFD_sent:
    for t2 in CFD_sent[t1]:
        MI_sent[t1][t2] =math.log(CFD_sent[t1][t2]*len(allTokens)/(freq[t1]*freq[t2]),2)

KeyboardInterrupt: 

In [ ]:
# conditional freq distribution and MI (using complete sentence)
CFD_sent = {}
MI_sent ={}
for word in allTokens:
    CFD_sent[word] = Counter()
    MI_sent[word] = {}
    
for sent in texts:
    for (index,token) in enumerate(sent):
        cfd_sent[token].update(sent)
#         if index > 4:
#             window = sent[index-5:index+5]   ## 前后5个单词,窗口
#         else:
#             window = sent[0:index+5]         ## 前后5个单词,窗口
#         cfd_win[token].update(window)       ## 增加入当前单词的相关单词

wordsCount = sum([len(text) for text in texts])
for t1 in CFD_sent:
    for t2 in CFD_sent[t1]:
        pmi_sent[t1][t2] =math.log(float(CFD_sent[t1][t2])*len(allTokens)/(fd[t1]*fd[t2]),2)
# for t1 in cfd_win:
#     for t2 in cfd_win[t1]:
#         cfd = float(cfd_win[t1][t2]+cfd_win[t2][t1])/2.0
#         pmi_win[t1][t2] = cfd/len(tokens)*math.log(cfd*len(tokens)/(fd[t1]*fd[t2]),2)
# for t1 in cfd_sent:
#     for t2 in cfd_sent[t1]:
#         pmi_sent[t1][t2] =math.log(float(cfd_sent[t1][t2])*len(tokens)/(fd[t1]*fd[t2]),2)

In our corpora, Mutual Information is calculated as follows:

>MI = log ( (AB * sizeCorpus) / (A * B * span) ) / log (2)

Suppose we are calculating the MI for the collocate color near purple in BYU-BNC.

A = frequency of node word (e.g. purple): 1262
B = frequency of collocate (e.g. color): 115
AB = frequency of collocate near the node word (e.g. color near purple): 24
sizeCorpus= size of corpus (# words; in this case the BNC): 96,263,399
span = span of words (e.g. 3 to left and 3 to right of node word): 6
log (2) is literally the log10 of the number 2: .30103

>MI = 11.37 = log ( (24 * 96,263,399) / (1262 * 115 * 6) ) / .30103

In [46]:
# -*- coding: utf-8 -*-
import re
import os
from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

path = 'D:\\nlp\\en\\COCA\\COCA_sample_lineartext\\'

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        if item not in ['has','was']:
            stemmed.append(stemmer.lemmatize(item))
    return ' '.join(stemmed)+'\n'

for filename in os.listdir(path+'raw'):
    with open(path+'raw\\'+filename) as f:
        doc = ""
        for sent in f.read().split(' . '):
            tokens = re.sub(r'<p>|(^| )[^a-z]+( |$)|[^a-z\'\.\-]+',' ',sent.lower()).split()#去<p>，保留字母和词中的 ' - .
            doc += stem_tokens(tokens,stemmer)
        doc = doc[:-1]

    with open(path+'result\\'+filename,'w') as f:
        f.write(doc[:-1])

In [28]:
import math
pmi_sent ={}
for i in tokens:
    pmi_sent[i] = {}

wordsCount = sum([len(text) for text in texts])
# for t1 in cfd_win:
#     for t2 in cfd_win[t1]:
#         cfd = float(cfd_win[t1][t2]+cfd_win[t2][t1])/2.0
#         pmi_win[t1][t2] = cfd/len(tokens)*math.log(cfd*len(tokens)/(fd[t1]*fd[t2]),2)
for t1 in cfd_sent:
    for t2 in cfd_sent[t1]:
        pmi_sent[t1][t2] =math.log(float(cfd_sent[t1][t2])*len(tokens)/(fd[t1]*fd[t2]),2)

In [14]:
threshold = 0.5
pmi_sent_filtered = {}
for i in pmi_sent:
    pmi_sent_filtered[i] = [(k,v) for (k, v) in pmi_sent[i].items() if v > threshold]

In [ ]:
target='penguin'

re = Counter()
for (token1,pmi1) in pmi_sent_filtered[target]:
    for (token2,pmi2) in pmi_sent_filtered[token1]:
        re.update({token2:pmi1*pmi2})

In [20]:
import time
t = time.time()
target='penguin'
re = Counter()
for (token1,pmi1) in pmi_sent_filtered[target]:
    for (token2,pmi2) in pmi_sent_filtered[token1]:
        tPMI = pmi1*pmi2
        for (token3,pmi3) in pmi_sent_filtered[token2]:
            re.update({token3:tPMI*pmi3})
re.most_common(50)
print(time.time()-t)

220.37415409088135


In [ ]:
re.most_common(50)
#re.most_common(len(filter(lambda v:v>50, re.values())))

In [84]:
set(x[0] for x in thirdpmi)-set(x[0] for x in cfd_sent['cloud'].most_common(650))
#set(thirdpmi)-set(re.viewkeys())

{'glass',
 'golden',
 'island',
 'maestro',
 'oil',
 'photo',
 'red',
 'rock',
 'sidebar',
 'skin',
 'small',
 'surface',
 'von'}

In [ ]:
import pandas as pd
pd.DataFrame(re,index=re.keys())

In [25]:
text = "I do not like green eggs and ham, I do not \"like\" them Sam I am!"
tokens = nltk.wordpunct_tokenize(text)
print(nltk.FreqDist(BigramCollocationFinder.from_words('12121234',window_size = 3)))

TypeError: 'BigramCollocationFinder' object is not iterable

In [72]:
from nltk.metrics.spearman import *
spearman_correlation(pmi_sent['se'],pmi_sent['handbook'])

0.9775185352641972

In [4]:
import nltk
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder = BigramCollocationFinder.from_words('121212341')
print finder.nbest(bigram_measures.pmi, 10)  
finder.apply_freq_filter(3)
print finder.nbest(bigram_measures.pmi, 10)
finder = BigramCollocationFinder.from_words('12121234',window_size = 3)
print finder.nbest(bigram_measures.pmi, 10)  

[('3', '4'), ('2', '3'), ('1', '2'), ('4', '1'), ('2', '1')]